# Part 1: Evaluate Multi-class Taggers with Precision, Recall, and F1-score 

This part has 15 points.

Study this notebook: run the code which trains different classifiers and evaluates average performances with precision, recall and f1score. The code also reports the scores for each individual label in each model.

Your task is to reimplement functions for accuracy_score (1pt), precision_score (3pt), recall_score (3pt), f1_score (2pt) with options for multiple labels and mico- and macro-averaging for them. The answer should be a simplified form of the function in scikit-learn ([precision_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html), [recall_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html), [f1_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)). 

For details check out [Chapter 4 section 4.7](https://web.stanford.edu/~jurafsky/slp3/4.pdf) (Jurafsty and Martin; 2019).

Add code to produce a confusion matrix (3pt)

Explain the effect of the backoff (3pt)


## Import Libraries

In [1]:
# Brown corpus:
import nltk
from nltk.corpus import brown

# load a tagger models
from nltk.tag.perceptron import PerceptronTagger
# Naive Bayes MLE 
from nltk.tag.sequential import NgramTagger

# tagset mapping:
from nltk.tag.mapping import map_tag

# plotting:
from matplotlib import pyplot as plt

# If you downloaded these corous before comment below lines
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

# you can compare you implementation with these
# evaluation metrics:
from sklearn.metrics import (
    f1_score as _f1_score, 
    precision_score as _precision_score, 
    recall_score as _recall_score,
    accuracy_score as _accuracy_score
)

import numpy as np

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\edina\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\edina\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\edina\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [2]:
# split training and testing data:
test_train_split = 500
test_set = brown.tagged_sents()[:test_train_split]
train_set = brown.tagged_sents()[test_train_split:]

## Prepare the training and testing dataset

## Load or train the classifiers

In [3]:
# load a pre-trained perceptron tagger:
perceptron_tagger = PerceptronTagger()

In [4]:
%%time
# train Naive Bayes / count-based ngram taggers:
unigram_tagger = NgramTagger(1, train=train_set)
bigram_tagger_nobackoff = NgramTagger(2, train=train_set)
bigram_tagger = NgramTagger(2, train=train_set, backoff=unigram_tagger)
trigram_tagger = NgramTagger(3, train=train_set, backoff=bigram_tagger)

Wall time: 36.4 s


In [5]:
models = {
    "Perceptron": perceptron_tagger, 
    "Unigram": unigram_tagger, 
    "Bigram": bigram_tagger, 
    "Trigram": trigram_tagger, 
    "Bigram-backoff": bigram_tagger_nobackoff, 
}

## Evaluate the models

The test dataset and the models are based on the English Penn TreeBank tagsets. However, we don't need that fine degree of granularity. Therefore, we map each tag onto unviversal tagset.

In [6]:
# the ground truth labels according to the dataset:
tags_true = [
    map_tag("en-brown", "universal", tag)
    for tagged_sent in test_set
    for word, tag in tagged_sent
]

# strip the tags:
test_set_sents = [
    [word for word, tag in tagged_sent]
    for tagged_sent in test_set
]

tagset = sorted(list(set(tags_true)))
print(tagset)

['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


In [7]:
def accuracy_score(y_true, y_pred):
    """
    y_true : 1d array-like Ground truth (correct) target values.
    y_pred : 1d array-like Estimated targets as returned by a classifier.
    FORMULA:
    ACC = correct / total
    """
    corr_count = 0
    tot = 0
    maxim = len(y_true)
    for i in range (maxim): #if find match for both true and pred incerement both else just increment max sum
        if y_pred[i] == y_true[i]:
            corr_count += 1
            tot += 1
        else:
            tot += 1
    accuracy = corr_count/tot
    # your code here
    #
    return accuracy# the overall accuracy score with exact match
    
def precision_score(y_true, y_pred, labels=None, average=None):
    """
    y_true : 1d array-like Ground truth (correct) target values.
    y_pred : 1d array-like Estimated targets as returned by a classifier.
    labels : list of unique labels for sort out the result
    average : string, ['micro', 'macro'] instead of defining labels.
    
    When true positive + false positive == 0 returns 0
    """
    maxim = len(y_true) #tot length of y_trues in order to later match them to tp or fp
    vals = {}

    for i in range(maxim):
        tag = y_pred[i]
        if tag not in vals:
            vals[tag] = {'true_positive': 0, 'false_positive': 0} #label tag of y_pred in y_true set labels
    
    for i in range(0, maxim):
        tag = y_pred[i]
        tag_true = y_true[i] #for every i-th item withing the y_ture  and y_pred test each with each other

        if tag == tag_true:
            vals[tag]['true_positive'] = vals[tag]['true_positive']+1
        elif tag != tag_true:
            vals[tag]['false_positive'] = vals[tag]['false_positive']+1

    if labels is None:
        if average == 'micro':
            #Mii matrix for dict with counts
            mini_mat = {'true_positive': 0, 'false_positive': 0}
            for key in vals:
                mini_mat['true_positive'] = mini_mat['true_positive'] + vals[key]['true_positive']
                mini_mat['false_positive'] = mini_mat['false_positive'] + vals[key]['false_positive']
            tp_counter = mini_mat['true_positive']
            fp_counter = mini_mat['false_positive']
            zero_div = tp_counter + fp_counter
            if zero_div == 0:
                micro_precision = 0
            else:
                micro_precision = tp_counter / zero_div
            return micro_precision # the score with micro averaging
        elif average == 'macro':
            #List List_prec --> Averaged
            list_prec = []
            for key in vals:
                tp_counter = vals[key]['true_positive']
                fp_counter = vals[key]['false_positive']
                zero_div = tp_counter + fp_counter
                if zero_div == 0:
                    precision = 0
                else:
                    precision = tp_counter / zero_div
                list_prec.append(precision)
            #Now average out proc.
            count_precs = 0
            for precision in list_prec:
                count_precs += precision
            zero_div = len(list_prec)
            macro_precision = count_precs / zero_div
            return macro_precision # the score with micro averaging
    else:
        #
        # your code here
        #
        prec_list = []
        for key in labels:
            tp_counter = vals[key]['true_positive']
            fp_counter = vals[key]['false_positive']
            zero_div = tp_counter + fp_counter
            if zero_div == 0:
                precision = 0
            else:
                precision = tp_counter / zero_div
            prec_list.append(precision)

        return prec_list# list of score of each label 

def recall_score(y_true, y_pred, labels=None, average=None):
    """
    y_true : 1d array-like Ground truth (correct) target values.
    y_pred : 1d array-like Estimated targets as returned by a classifier.
    labels : list of unique labels for sort out the result
    average : string, ['micro', 'macro'] instead of defining labels.
    
    When true positive + false positive == 0 returns 0
    """

    maxim = len(y_true)
    vals = {}
    #the rest of the structure below will follow the same pattern, assigning to tags with labels tp and fn
    #comparing the tags and finally dealing with zero divsion issues, with zero_div
    for i in range (0, maxim):
        tag = y_pred[i]
        if tag not in vals:
            vals[tag] = {'true_positive': 0, 'false_negative': 0}
    
    for i in range (0, maxim):
        tag = y_pred[i]
        tag_true = y_true[i]

        if tag == tag_true:
            vals[tag]['true_positive'] = vals[tag]['true_positive'] + 1
        elif tag != tag_true:
            vals[tag_true]['false_negative'] = vals[tag_true]['false_negative'] + 1


    if labels is None:
        if average == 'micro':
            #create mini matrix for tp and fn aka sum(tp + fn) but tags
            mat = {'true_positive': 0, 'false_negative': 0}
            for key in vals:
                mat['true_positive'] = mat['true_positive'] + vals[key]['true_positive']
                mat['false_negative'] = mat['false_negative'] + vals[key]['false_negative']
            tp_counter = mat['true_positive']
            fn_counter = mat['false_negative']
            zero_div = tp_counter + fn_counter
            if zero_div == 0:
                micro_recall = 0
            else:
                micro_recall = tp_counter / zero_div
            return micro_recall # the score with micro averaging
        elif average == 'macro':
            comp_list_rec = []
            for key in vals:
                tp_counter = vals[key]['true_positive']
                fn_counter = vals[key]['false_negative']
                zero_div = tp_counter + fn_counter
                if zero_div == 0:
                    m_recall = 0
                else:
                    m_recall = tp_counter / zero_div
                comp_list_rec.append(m_recall)
            count_recs = 0
            for m_recall in comp_list_rec:
                count_recs += m_recall
            zero_div = len(comp_list_rec)
            macro_recall = count_recs / zero_div
            #
            # your code here
            #
            return macro_recall # the score with macro averaging
    else:
        #
        # your code here
        #
        recs_list = []
        for key in labels:
            tp_counter = vals[key]['true_positive']
            fn_counter = vals[key]['false_negative']
            zero_div = tp_counter + fn_counter
            if zero_div == 0:
                recall = 0
            else:
                recall = tp_counter / zero_div
            recs_list.append(recall)
        return recs_list # list of score of each label 

def f1_score(y_true, y_pred, labels=None, average=None):
    """
    y_true : 1d array-like Ground truth (correct) target values.
    y_pred : 1d array-like Estimated targets as returned by a classifier.
    labels : list of unique labels for sort out the result
    average : string, ['micro', 'macro'] instead of defining labels.

    The F1 score has its best score at one and bad value at 0.
    Formula for F1:
    F1 = 2 * (precision * recall) / (precision + recall)
    """
    # you can call recall_score and precision_score.
    if labels is None:
        if average == 'micro':
            #
            micro_pre = precision_score(y_true, y_pred, labels=None, average='micro')
            micro_rec = recall_score(y_true, y_pred, labels=None, average='micro')
            zero_div = micro_pre + micro_rec
            if zero_div == 0:
                f1 = 0
            else:
                f1 = (2 * (micro_pre * micro_rec) / zero_div)

            return f1   # the score with micro averaging
        
        elif average == 'macro':
            # your code here
            macro_pre = precision_score(y_true, y_pred, labels=None, average='macro')
            macro_rec = recall_score(y_true, y_pred, labels=None, average='macro')
            zero_div = macro_pre + macro_rec
            if zero_div == 0:
                f1 = 0
            else:
                f1 = (2 * (macro_pre * macro_rec) / zero_div)
            return f1     # the score with micro averaging
    else:
        #
        # Score of F1 to assign to list
        f1_scores = []
        #Calls for funcs.
        prec_list = precision_score(y_true, y_pred, labels=None, average='None')
        rec_list = recall_score(y_true, y_pred, labels=None, average='None')
        dist = len(f1_scores)
        for i in range (0, dist):
            precision = prec_list[i]
            recall = rec_list[i]
            zero_div = precision + recall
            if zero_div == 0:
                f1 = 0
            else:
                f1 = (2 * (precs * recs) / zero_div)
            f1_scores.append(f1)
        return f1_scores     # the score with micro averaging          # list of score of each label 
    

def all_metrics(y_true, y_pred, labels=None, average=None):
    # you can compare you implementation with these
    return (
        precision_score(y_true, y_pred, labels=labels, average=average),
        recall_score(y_true, y_pred, labels=labels, average=average),
        f1_score(y_true, y_pred, labels=labels, average=average),
        accuracy_score(y_true, y_pred)
    )
    # remove the likes above and use the function calls below: 
    #return (
       # precision_score(y_true, y_pred, labels=labels, average=average),
        #recall_score(y_true, y_pred, labels=labels, average=average),
        #f1_score(y_true, y_pred, labels=labels, average=average),
        #accuracy_score(y_true, y_pred)
    #)


In [8]:
models_preds = dict()
print("              |       |         macro       |         micro")
print("  model name  |  acc  | preci  recal    f1  | preci  recal    f1")
print("-"*58)
for model_name, model in models.items(): #We will use this data from models() to output ate rin matrix
    tags_pred = [
        map_tag("en-ptb", "universal", tag) if model_name == "Perceptron" else map_tag("en-brown", "universal", tag)
        for sent in test_set_sents
        for word, tag in model.tag(sent)
    ]
    models_preds[model_name] = tags_pred
    # print the results
    
    precision_macro, recall_macro, f1score_macro, accuracy = all_metrics(tags_true, tags_pred, average='macro')
    precision_micro, recall_micro, f1score_micro, _ = all_metrics(tags_true, tags_pred, average='micro')
    print(f"{model_name:14}| {100*accuracy:5.2f} | {100*precision_macro:5.2f}  {100*recall_macro:5.2f}  {100*f1score_macro:5.2f} | {100*precision_micro:5.2f}  {100*recall_micro:5.2f}  {100*f1score_micro:5.2f}")
    

              |       |         macro       |         micro
  model name  |  acc  | preci  recal    f1  | preci  recal    f1
----------------------------------------------------------
Perceptron    | 93.66 | 86.17  88.79  87.46 | 93.66  93.66  93.66
Unigram       | 93.30 | 86.11  94.57  90.14 | 93.30  93.30  93.30
Bigram        | 94.40 | 87.36  94.11  90.61 | 94.40  94.40  94.40
Trigram       | 94.54 | 87.53  94.48  90.87 | 94.54  94.54  94.54
Bigram-backoff| 24.63 | 87.72  31.96  46.85 | 24.63  24.63  24.63


In [9]:
for model_name, tags_pred in models_preds.items(): #and we will iterate this data and output it
    print('='*50)
    print(model_name) 
    print('')
    precisions, recalls, f1scores, _ = all_metrics(tags_true, tags_pred, labels=tagset)
    print("tag\tprecision\trecall\tf1-score")
    print("-"*50)
    for tag, precision, recall, f1score in zip(tagset, precisions, recalls, f1scores):
        print(f"{tag}\t{100*precision:9.2f}\t{100*recall:6.2f}\t{100*f1score:8.2f}")
    print('='*50)

Perceptron

tag	precision	recall	f1-score
--------------------------------------------------
Unigram

tag	precision	recall	f1-score
--------------------------------------------------
Bigram

tag	precision	recall	f1-score
--------------------------------------------------
Trigram

tag	precision	recall	f1-score
--------------------------------------------------
Bigram-backoff

tag	precision	recall	f1-score
--------------------------------------------------


In [10]:
# Add confusion matrix here CM taken from nltk site
from nltk.metrics import ConfusionMatrix
for model_name, tags_pred in models_preds.items():
    print(model_name + '\n')
    ref = tags_true 
    test = tags_pred
    CM = ConfusionMatrix(ref, test)
    print(CM)

Perceptron

     |                        C         N         P         V      |
     |         A    A    A    O    D    O    N    R    P    E      |
     |         D    D    D    N    E    U    U    O    R    R      |
     |    .    J    P    V    J    T    N    M    N    T    B    X |
-----+-------------------------------------------------------------+
   . |<1226>   1    .    .    .    .    .    .    .    2    .    . |
 ADJ |    . <638>   1   13    .    .  120   18    .    .    5    . |
 ADP |    .    2<1359>   8    .    5    .    .    .  119    7    . |
 ADV |    .   14   12 <302>   1    2    .    .    1    2    1    1 |
CONJ |    .    .    .    . <253>   1    .    .    .    .    .    . |
 DET |    .    .    1    1    .<1265>   1    .  102    .    .    . |
NOUN |    .   64    .    2    .    .<3464>   7    1    .   22    1 |
 NUM |    .    4    .    .    .    .    . <206>   .    .    .    . |
PRON |    .    .    1    .    .    7    .    . <273>   .    .    . |
 PRT |    .    .    9 

## Backoff

Compare the result of the bigram models (with and without backoff). In one or two sentences, explain why backoff is important for the n-gram models? Why tagging works better with backoff?

For more details check out the definition of backoff in [Chapter 3 section 3.4.3](https://web.stanford.edu/~jurafsky/slp3/3.pdf) (Jurafsty and Martin; 2019).

In [11]:
#We can estimate the probabilities of unseen n-grams instead of throwing us errors when we test larger n-grams we can use Backoff which includes
#smoothing, so say we want a trigram for rare events... 
#Good this about Backoof, it uses less context if bigram, unigram etc is sufficient enough hence "back"..."off".
#Less context in this case is better for the things the model hasnt learned yet. Consequently this means taht it will have a bigger dataset which
#will lead to less mistakes when tagging. Therefore it is important since it make up for the insuffiecny of the rare cases.
#Comparing the results (Bigram and Bigram-backoff) we can see that "no backoff" has good enough precision score but a very low recall
#score thus it will give higher count of false negatives.

#So shorter answear:
#If zero occurences of N-gram equls true when not using backoff, the proability will also be zero and no tag can be assigned.
#Therefore not tag can be correctly predicted for some rare N-grams.
#With backoff, if there is an instance of rare N-gram whcih hasnt been seen it will "backoff" to N-1 -gram and 
#then we will still get a chance to guess the crrect label even if specific bi-gram hasnt been seen. So it 
#fall to lesser context but still has a valid guessing spektrum.